In [1]:
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

from datasets.rsna_breast_cancer import BreastCancerDataset
from simplenet import SimpleNet
from src import backbones

/Users/ksoll/Documents/git/SimpleNet/venv/lib/python3.11/site-packages/torchvision/transforms/functional_pil.py:5: UserWarning: The torchvision.transforms.functional_pil module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [2]:
device = 'mps'

In [3]:
backbone = backbones.load("resnet50")
trained_network = SimpleNet(device)
ckpt_path = "/Users/ksoll/Documents/git/SimpleNet/models/test"
trained_network.load_model(ckpt_path, "ckpt_epoch_59.pth", backbone)

/Users/ksoll/Documents/git/SimpleNet/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ksoll/Documents/git/SimpleNet/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [53]:
ds = BreastCancerDataset(
    img_dir="/Users/ksoll/Documents/git/SimpleNet/data/rsna_breast_cancer",
    meta_data_csv_path="/Users/ksoll/Documents/git/SimpleNet/train.csv",
    num_images=(1024, 10000, 128, 0),
    imagesize=(128, 128)
)

dataloader = DataLoader(ds, batch_size=4)

true_labels = []
trained_network_pred = []

for data in tqdm(dataloader):
    true_labels.extend(data['anomaly'].cpu().numpy())
    trained_pred, _, _ = trained_network.predict(data['image'])

    trained_network_pred.extend(trained_pred)

true_labels = np.array(true_labels)
trained_network_pred = np.array(trained_network_pred)

img_min_scores = trained_network_pred.min(axis=-1)
img_max_scores = trained_network_pred.max(axis=-1)
trained_network_pred = (trained_network_pred - img_min_scores) / (img_max_scores - img_min_scores)

del trained_network, backbone
torch.cuda.empty_cache()

100%|██████████| 288/288 [03:01<00:00,  1.59it/s]


In [56]:
backbone = backbones.load("resnet50")
untrained_net = SimpleNet(device)
untrained_net.load(
    backbone=backbone,
    layers_to_extract_from=['layer2', 'layer3'],
    device=device,
    input_shape=(3, 128, 128),
    pretrain_embed_dimension=32,
    target_embed_dimension=16,
    patchsize=9,
    embedding_size=None,
    meta_epochs=1,
    aed_meta_epochs=1,
    gan_epochs=1,
    noise_std=0.4,
    dsc_layers=4,
    dsc_hidden=8,
    dsc_margin=0.25,
    dsc_lr=0.001,
    train_backbone=True,
    cos_lr=False,
    lr=0.001,
    pre_proj=1,
    proj_layer_type=0,
    mix_noise=1,
)

In [57]:
untrained_network_pred = []

for data in tqdm(dataloader):
    untrained_pred, _, _ = untrained_net.predict(data['image'])

    untrained_network_pred.extend(untrained_pred)

untrained_network_pred = np.array(untrained_network_pred)

img_min_scores = untrained_network_pred.min(axis=-1)
img_max_scores = untrained_network_pred.max(axis=-1)
untrained_network_pred = (untrained_network_pred - img_min_scores) / (img_max_scores - img_min_scores)

del untrained_net, backbone
torch.cuda.empty_cache()

100%|██████████| 288/288 [02:53<00:00,  1.66it/s]


In [58]:
#Random

f1s = []
aurocs = []

random_predictions = np.random.uniform(size=(1000, len(ds)))

for idx in range(1000):
    random_preds = random_predictions[idx, :]
    random_binary = np.where(random_preds >= 0.5, 1, 0)
    f1 = f1_score(true_labels, random_binary)
    auroc = roc_auc_score(true_labels, random_preds)

    f1s.append(f1)
    aurocs.append(auroc)

print("Random")
print(f"F1: {np.mean(f1s)}, Auroc: {np.mean(aurocs)}")

Random
F1: 0.18149958847937314, Auroc: 0.49969757843017576


In [59]:
#Untrained Network
untrained_binary = np.where(untrained_network_pred >= 0.5, 1, 0)
f1 = f1_score(true_labels, untrained_binary)
auroc = roc_auc_score(true_labels, untrained_network_pred)

print("Untrained")
print(f"F1: {f1}, Auroc: {auroc}")

Untrained
F1: 0.2134387351778656, Auroc: 0.5386123657226562


In [60]:
#Trained Network
trained_binary = np.where(trained_network_pred >= 0.5, 1, 0)
f1 = f1_score(true_labels, trained_binary)
auroc = roc_auc_score(true_labels, trained_network_pred)

print("Trained")
print(f"F1: {f1}, Auroc: {auroc}")

Trained
F1: 0.1764705882352941, Auroc: 0.5199012756347656
